# Faturamento Mensal 

In [0]:
USE olist.Gold;

-- Pergunta: Faturamento mensal
SELECT
  -- 'date_trunc' é uma função fantástica para agrupar por mês/semana/dia
  DATE_TRUNC('month', order_purchase_timestamp) AS mes,
  SUM(total_payment_value) AS faturamento_total_mensal,
  COUNT(DISTINCT order_id) AS total_pedidos
FROM
  fct_sales
WHERE
  order_status NOT IN ('CANCELED', 'UNAVAILABLE')
GROUP BY
  mes
ORDER BY
  mes ASC;

mes,faturamento_total_mensal,total_pedidos
2016-09-01T00:00:00.000Z,272.46,2
2016-10-01T00:00:00.000Z,67668.71000000005,290
2016-12-01T00:00:00.000Z,19.62,1
2017-01-01T00:00:00.000Z,187534.37999999974,787
2017-02-01T00:00:00.000Z,341415.86,1718
2017-03-01T00:00:00.000Z,520460.9500000006,2617
2017-04-01T00:00:00.000Z,488685.6200000008,2377
2017-05-01T00:00:00.000Z,718240.450000002,3640
2017-06-01T00:00:00.000Z,597442.4700000036,3205
2017-07-01T00:00:00.000Z,729875.3100000005,3946


Databricks visualization. Run in Databricks to view.

# Quais os 5 Estados que mais compram?

In [0]:
SELECT
  c.customer_state AS estado,
  SUM(f.total_payment_value) AS faturamento_total
FROM
  fct_sales AS f
INNER JOIN
  dim_customers AS c ON f.customer_id = c.dim_customer_id
WHERE
  f.order_status NOT IN ('CANCELED', 'UNAVAILABLE')
GROUP BY
  estado
ORDER BY
  faturamento_total DESC
LIMIT 5;

estado,faturamento_total
SP,7544659.299999849
RJ,2753911.4299999774
MG,2308330.7900000056
RS,1128464.0099999944
PR,1049730.7399999942


Databricks visualization. Run in Databricks to view.

# Quais as 10 categorias mais vendidas? 

In [0]:
SELECT
  p.product_category_name_english AS categoria_produto,
  COUNT(DISTINCT f.order_id) AS total_pedidos,
  SUM(f.total_payment_value) AS faturamento_total
FROM
  fct_sales AS f
INNER JOIN
  dim_products AS p ON f.product_id = p.dim_product_id
WHERE
  f.order_status NOT IN ('CANCELED', 'UNAVAILABLE')
  AND p.product_category_name_english IS NOT NULL
GROUP BY
  categoria_produto
ORDER BY
  faturamento_total DESC
LIMIT 10;

categoria_produto,total_pedidos,faturamento_total
BED_BATH_TABLE,9399,1711258.0800000029
HEALTH_BEAUTY,8800,1653730.4500000011
COMPUTERS_ACCESSORIES,6654,1571543.8099999996
FURNITURE_DECOR,6425,1424782.5200000023
WATCHES_GIFTS,5604,1421715.2800000017
SPORTS_LEISURE,7673,1381363.2300000007
HOUSEWARES,5847,1086565.3200000005
AUTO,3872,843297.6500000004
GARDEN_TOOLS,3505,823517.7999999997
COOL_STUFF,3616,759644.8500000008


Databricks visualization. Run in Databricks to view.

# Quais meses o faturamento foi reduzido?

In [0]:
-- Query para plotar a variação percentual de TODOS os meses

WITH faturamento_mensal AS (
  SELECT
    DATE_TRUNC('month', order_purchase_timestamp) AS mes,
    SUM(total_payment_value) AS faturamento_total_mensal
  FROM
    olist.Gold.fct_sales
  WHERE
    order_status NOT IN ('CANCELED', 'UNAVAILABLE')
  GROUP BY
    mes
),

calculo_diferenca AS (
  SELECT
    mes,
    faturamento_total_mensal,
    LAG(faturamento_total_mensal, 1) OVER (ORDER BY mes) AS faturamento_mes_anterior,
    
    (faturamento_total_mensal - LAG(faturamento_total_mensal, 1) OVER (ORDER BY mes)) * 100.0 / LAG(faturamento_total_mensal, 1) OVER (ORDER BY mes) AS variacao_percentual
  FROM
    faturamento_mensal
)


SELECT
  mes,
  faturamento_total_mensal,
  faturamento_mes_anterior,
  ROUND(variacao_percentual, 2) AS variacao_percentual_arredondada
FROM
  calculo_diferenca
ORDER BY
  mes ASC;

mes,faturamento_total_mensal,faturamento_mes_anterior,variacao_percentual_arredondada
2016-09-01T00:00:00.000Z,272.46,null,null
2016-10-01T00:00:00.000Z,67668.71000000005,272.46,24736.2
2016-12-01T00:00:00.000Z,19.62,67668.71000000005,-99.97
2017-01-01T00:00:00.000Z,187534.3799999996,19.62,955732.72
2017-02-01T00:00:00.000Z,341415.8600000005,187534.3799999996,82.06
2017-03-01T00:00:00.000Z,520460.95000000036,341415.8600000005,52.44
2017-04-01T00:00:00.000Z,488685.620000001,520460.95000000036,-6.11
2017-05-01T00:00:00.000Z,718240.450000003,488685.620000001,46.97
2017-06-01T00:00:00.000Z,597442.4700000042,718240.450000003,-16.82
2017-07-01T00:00:00.000Z,729875.3100000008,597442.4700000042,22.17


Databricks visualization. Run in Databricks to view.